# Projet Final Apache Spark

**Nom Etudiant :  SARR**  

**Prenom Etudiant: Abdoulaye**  

**Classe : BIG DATA**  


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public ou privé) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Jeudi 10 janvier 2021** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [1]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://abdoulayesarr-HP-EliteBook-8460p:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1610225619210)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [2]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [3]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [4]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


In [5]:
fireDF.select("Zipcode","Box").show(2)

+-------+----+
|Zipcode| Box|
+-------+----+
|  94109|3362|
|  94124|6495|
+-------+----+
only showing top 2 rows



Nous allons mettre en cache le Dataframe

In [6]:
fireDF.cache()

res1: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [7]:
fireDF.count()

res2: Long = 175296


In [8]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [9]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [10]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [11]:
val rst = fewFireDF.select($"CallType").distinct().count()


rst: Long = 29


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [12]:
val result = fewFireDF.
             select($"CallType").
             filter($"CallType".contains("Fire")).distinct().
             show()

+--------------+
|      CallType|
+--------------+
|   Marine Fire|
|  Vehicle Fire|
|  Outside Fire|
|Structure Fire|
+--------------+



result: Unit = ()


### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [13]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
val rst = newFireDF
         .select("ResponseDelayedinMins", "AvailableDtTm","CallType","Neighborhood")
         .filter($"ResponseDelayedinMins" > 5)
         .show(10)

+---------------------+--------------------+--------------------+--------------------+
|ResponseDelayedinMins|       AvailableDtTm|            CallType|        Neighborhood|
+---------------------+--------------------+--------------------+--------------------+
|                 5.35|01/11/2002 04:34:...|    Medical Incident|  West of Twin Peaks|
|                 6.25|01/12/2002 01:23:...|    Medical Incident|        Russian Hill|
|                  5.2|01/13/2002 01:51:...|    Medical Incident|          Tenderloin|
|                  5.6|01/14/2002 08:16:...|Citizen Assist / ...|     Sunset/Parkside|
|                 7.25|01/14/2002 12:27:...|    Medical Incident|Financial Distric...|
|            11.916667|01/15/2002 06:53:...|    Medical Incident|     South of Market|
|             5.116667|01/15/2002 03:18:...|    Medical Incident| Castro/Upper Market|
|             8.633333|01/15/2002 03:55:...|    Medical Incident|         North Beach|
|             95.28333|01/15/2002 07:49:...

newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
rst: Unit = ()


### Transformations des dates
Maintenant nous allons d'abord:
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard
2. Retourner le Dataframe transformée
3. Mettre en cache le nouveau DataFrame

In [14]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res6: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [15]:
fireTSDF.select($"OnWatchDate",$"AvailableDtTS",$"IncidentDate").show()

+-------------------+-------------------+-------------------+
|        OnWatchDate|      AvailableDtTS|       IncidentDate|
+-------------------+-------------------+-------------------+
|2002-01-10 00:00:00|2002-01-11 01:51:44|2002-01-11 00:00:00|
|2002-01-10 00:00:00|2002-01-11 03:01:18|2002-01-11 00:00:00|
|2002-01-10 00:00:00|2002-01-11 02:39:50|2002-01-11 00:00:00|
|2002-01-10 00:00:00|2002-01-11 04:16:46|2002-01-11 00:00:00|
|2002-01-10 00:00:00|2002-01-11 06:01:58|2002-01-11 00:00:00|
|2002-01-11 00:00:00|2002-01-11 08:03:26|2002-01-11 00:00:00|
|2002-01-11 00:00:00|2002-01-11 09:46:44|2002-01-11 00:00:00|
|2002-01-11 00:00:00|2002-01-11 09:58:53|2002-01-11 00:00:00|
|2002-01-11 00:00:00|2002-01-11 12:06:57|2002-01-11 00:00:00|
|2002-01-11 00:00:00|2002-01-11 13:08:40|2002-01-11 00:00:00|
|2002-01-11 00:00:00|2002-01-11 15:31:02|2002-01-11 00:00:00|
|2002-01-11 00:00:00|2002-01-11 14:59:04|2002-01-11 00:00:00|
|2002-01-11 00:00:00|2002-01-11 16:22:49|2002-01-11 00:00:00|
|2002-01

### Question 4
**Quels sont les types d'appels les plus courants?**

In [87]:
//Reponse 4
val appelCourant = fireTSDF.select("CallType").
                  groupBy("CallType").count().
                  orderBy(desc("count"))
//affichons les dix types d'appel les plus courants
appelCourant.show(10)

+--------------------+------+
|            CallType| count|
+--------------------+------+
|    Medical Incident|113794|
|      Structure Fire| 23319|
|              Alarms| 19406|
|   Traffic Collision|  7013|
|Citizen Assist / ...|  2524|
|               Other|  2166|
|        Outside Fire|  2094|
|        Vehicle Fire|   854|
|Gas Leak (Natural...|   764|
|        Water Rescue|   755|
+--------------------+------+
only showing top 10 rows



appelCourant: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string, count: bigint]


la reponse prouve que Medical Incident est largement plus courant

### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [90]:
//Reponse 5-a
val appelCourant = fireTSDF.//select("CallType").
                  groupBy($"CallType",$"Box").count().
                  orderBy(desc("count"))
//affichons les 10 Box les plus courants
appelCourant.show(10)



+----------------+----+-----+
|        CallType| Box|count|
+----------------+----+-----+
|Medical Incident|2251| 1544|
|Medical Incident|1453| 1491|
|Medical Incident|1461| 1205|
|Medical Incident|5236| 1015|
|Medical Incident|1545|  933|
|Medical Incident|2252|  913|
|Medical Incident|2248|  873|
|Medical Incident|1365|  822|
|Medical Incident|1546|  818|
|Medical Incident|2242|  816|
+----------------+----+-----+
only showing top 10 rows



appelCourant: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string, Box: string ... 1 more field]


### Question 5-b
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [ ]:
//Reponse 5-b
val fewFireDF = fireDF.select("Neighborhood", "ZipCode").
                where($"ZipCode" === 94102 || $"ZipCode" === 94103).
                distinct()
fewFireDF.show()

### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [92]:
//Reponse 6
// on utilise la fonction describe pour voir les statistiques sur les temps de reponses
fireDF.select("Delay").describe().show()

+-------+-----------------+
|summary|            Delay|
+-------+-----------------+
|  count|           175296|
|   mean|3.892364154521585|
| stddev|9.378286226254206|
|    min|      0.016666668|
|    max|          1844.55|
+-------+-----------------+



### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

### affichons les années distincts 

In [94]:
val annees = fireTSDF.select(year($"IncidentDate")).distinct
annees.show()

+------------------+
|year(IncidentDate)|
+------------------+
|              2003|
|              2007|
|              2018|
|              2015|
|              2006|
|              2013|
|              2014|
|              2004|
|              2012|
|              2009|
|              2016|
|              2001|
|              2005|
|              2000|
|              2010|
|              2011|
|              2008|
|              2017|
|              2002|
+------------------+



annees: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [year(IncidentDate): int]


###  Maintenant on calcule le nombre d'annees distinctes avec la fonction count()

In [95]:

val nbrDateDistinct = annees.count()
print(s"le nombre d'années distinctes est: $nbrDateDistinct")

le nombre d'années distinctes est: 19

nbrDateDistinct: Long = 19


### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

Recupérons les enregistrement 2018 et leurs semaine

In [20]:
//Reponse 7-b
val enreg2018AndWeek = fireTSDF.select(year($"IncidentDate").as("date"),
                weekofyear(col("IncidentDate")).as("weekofyear")).
                filter($"date"==="2018")
enreg2018AndWeek.show()

+----+----------+
|date|weekofyear|
+----+----------+
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
|2018|         3|
+----+----------+
only showing top 20 rows



enreg2018AndWeek: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: int, weekofyear: int]


##### groupons les enregistrement par les differentes semaine avec groupby-> count-> orderby
   afin de recuperer les enregistrements des semaines les plus courant
 

In [21]:
val groupeByWeek  = enreg2018AndWeek.groupBy($"weekofyear").
            count().orderBy(desc("count"))
groupeByWeek.show(5)

+----------+-----+
|weekofyear|count|
+----------+-----+
|        22|  259|
|        40|  255|
|        43|  250|
|        25|  249|
|         1|  246|
+----------+-----+
only showing top 5 rows



groupeByWeek: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [weekofyear: int, count: bigint]


#### on voit clairement que la semaine 22 à plus d'appels donc on recupere cette premiere enregistrement avec la fonction take et map

In [22]:
groupeByWeek.take(1)
.map(a => println(s"la semaine ${a(0)} a plus d'appel avec: ${a(1)}   enregistrements"))

la semaine 22 a plus d'appel avec: 259   enregistrements


res13: Array[Unit] = Array(())


### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

Recupérons les enrégistrements de 2018, les quartiers et les délais

In [50]:
val enreg2018 = fireTSDF.select(year($"IncidentDate").
                as("date"),$"ResponseDelayedinMins",$"Neighborhood").
                filter($"date"==="2018")
enreg2018.show(10)

+----+---------------------+--------------------+
|date|ResponseDelayedinMins|        Neighborhood|
+----+---------------------+--------------------+
|2018|            2.8833334|    Presidio Heights|
|2018|            6.3333335|         Mission Bay|
|2018|                 2.65|           Chinatown|
|2018|            3.5333333|Financial Distric...|
|2018|                  1.1|          Tenderloin|
|2018|                 4.05|Bayview Hunters P...|
|2018|            2.5666666|      Inner Richmond|
|2018|                  1.4|        Inner Sunset|
|2018|            2.6666667|     Sunset/Parkside|
|2018|            1.7666667|     South of Market|
+----+---------------------+--------------------+
only showing top 10 rows



enreg2018: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: int, ResponseDelayedinMins: float ... 1 more field]


 on regroupe les enrégistrements par quartier puis on fait la sommes totales de leurs délai qui représente le temps de réponse grace aux fonction groupby agg sum et orderBy(pour trier par ordre decroissant)
 et on recupere les 10 quartiers ou il y a plus de temps de réponse

In [54]:
//Reponse 8
val neighborhood = enreg2018
         .groupBy($"Neighborhood")
         .agg(sum($"ResponseDelayedinMins").as("delai_total"))
         .orderBy(desc("delai_total"))

neighborhood.show(10)

+--------------------+------------------+
|        Neighborhood|       delai_total|
+--------------------+------------------+
|          Tenderloin| 5713.416682377458|
|     South of Market| 4019.916673846543|
|Financial Distric...|3353.6333242356777|
|             Mission|3150.3333284556866|
|Bayview Hunters P...|2411.9333442747593|
|     Sunset/Parkside|1240.1333360522985|
|           Chinatown|1182.3499933183193|
|    Western Addition|1156.0833313167095|
|            Nob Hill|1120.9999947845936|
|        Hayes Valley| 980.7833325713873|
+--------------------+------------------+
only showing top 10 rows



neighborhood: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Neighborhood: string, delai_total: double]


### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [ ]:
//Reponse 9
fewFireDF.write.parquet("fewFireDF.parquet")

### Question 10
**Comment relire les données stockée en format Parquet?**

In [ ]:
//Reponse 10
spark.read.parquet("fewFireDF.parquet").show()

## FIN